In [1]:
import tkinter as tk
from tkinter import ttk
from functions import handle_clean_text
import pickle
from textblob import TextBlob

with open("tfidf_vectorizer.pkl", "rb") as f:
    tfidf = pickle.load(f)

with open("logistic_regression_model.pkl", "rb") as f:
    model = pickle.load(f)
    
class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Sentiment App")
        self.geometry("600x800")
        self.configure(background="#ededed") 
        self.sentiment = ""
        self.resultColor = "blue"
        self.create_widgets()
        
    def create_widgets(self):
        # Title Section
        title_section = ttk.Frame(self, width=800)
        title_section.pack(fill="x", pady=10)
        title_label = ttk.Label(title_section, text="Text Detector System", font=("Arial", 20))
        title_label.pack()
        
        # Input Section
        input_section = ttk.Frame(self, width=800)
        input_section.pack(fill="x", pady=10)
        input_label = ttk.Label(input_section, text="Enter your text for sentiment analysis:", font=("Arial", 14))
        input_label.pack(anchor="w", padx=10, pady=5)
        
        # Text Box for User Input
        self.user_input = tk.Text(input_section, height=10, width=50, font=("Arial", 12))
        self.user_input.pack(fill="x", padx=10, pady=5)
        
        # Submit Button
        submit_button = ttk.Button(input_section, text="Analyze Text", command=self.handle_analyse_text)
        submit_button.pack(pady=10, padx=5)
        
        # Result Section to Display Output
        self.result_section = ttk.Frame(self, width=800)
        self.result_section.pack(fill="x", pady=10)
        
        # Label to show the result text
        self.result_label = ttk.Label(self.result_section, text="", font=("Arial", 14))
        self.result_label.pack(padx=10, pady=5)
        
    def handle_analyse_text(self):
        user_text = self.user_input.get("1.0", tk.END)
        cleaned_text = handle_clean_text([user_text])
        input_tfidf = tfidf.transform(cleaned_text)
        prediction = model.predict(input_tfidf.toarray())
        if prediction[0] == 1:
            self.sentiment = "Neutral"
            self.resultColor = "blue"
        elif prediction[0] == 2:
            self.sentiment = "Positive"
            self.resultColor = "green"
        elif prediction[0] == 3:
            self.sentiment = "Negative"
            self.resultColor = "red"
        else:
            self.sentiment = "Unknown"
            
        blob, polarity, subjectivity = self.handle_calculate_polarity_subjectivity(user_text)
        self.result_label.config(text=f"Result: {self.sentiment} (Polarity: {polarity}%, Subjectivity: {subjectivity}%)", foreground=self.resultColor) 
    
    def handle_calculate_polarity_subjectivity(self, user_text):
        blob = TextBlob(user_text)
        polarity = blob.sentiment.polarity * 100
        subjectivity = blob.sentiment.subjectivity * 100
        return blob, polarity, subjectivity
if __name__ == "__main__":
    app = App() 
    app.mainloop()
